# Gradient descent opdracht

Dit is mijn uitwerking van de Gradient descent implementatie opdracht. Als uitbreiding heb ik gekozen voor het normaliseren van de input-variabelen.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

## LinearRegression klasse

In [2]:
class MyLinearRegression:

    def __init__(self):
        self.w = 1.0
        self.b = 0.0
        self.x_mean = 0
        self.x_std = 1
        self.y_mean = 0
        self.y_std = 1

    def fit(self, feature, target, w0=1.0, b0=0.0, learn_rate=0.01, max_steps=1000):
        """
        Traint het lineaire regressiemodel.

        Parameters:
        feature (pd.DataFrame): DataFrame met 1 feature-kolom
        target (np.ndarray): Numpy array met target-kolom
        w0 (float): Startwaarde voor w
        b0 (float): Startwaarde voor b
        learn_rate (float): Learning rate
        max_steps (int): Maximum aantal iteraties
        """
        self.w = w0
        self.b = b0

        # pak de rijen van de kolom
        x = feature.iloc[:, 0].values
        y = target
        n = len(y)

        # normaliseren
        self.x_mean = np.mean(x)
        self.x_std = np.std(x)
        x_norm = (x - self.x_mean) / self.x_std

        self.y_mean = np.mean(y)
        self.y_std = np.std(y)
        y_norm = (y - self.y_mean) / self.y_std

        for step in range(max_steps):
            # maak voorspellingen
            y_voorspellingen = self.w * x_norm + self.b

            # afwijkingen berekenen
            errors = y_voorspellingen - y_norm

            # gradients berekenen met afgeleide MSE
            dw = (2/n) * np.sum(errors * x_norm)
            db = (2/n) * np.sum(errors)

            # update w en b
            self.w -= learn_rate * dw
            self.b -= learn_rate * db
    
    def predict(self, x):
        """
        Maakt voorspelling(en) met het lineaire regressiemodel.

        Parameters:
        x (int, float, pd.DataFrame): Een enkele observatie, of meerdere in een DataFrame (met 1 kolom).
        
        Returns:
        np.ndarray: De gemaakte voorspelling(en).
        """
        if isinstance(x, pd.DataFrame):
            # pak de rijen van de kolom
            x_values = x.iloc[:, 0].values
        else:
            # de input is een enkel getal, zet het in een np array
            x_values = np.array([x])

        # normaliseren
        x_norm = (x_values - self.x_mean) / self.x_std
        
        # doe de voorspelling
        y_norm_voorspelling = self.w * x_norm + self.b

        # converteren naar originele schaal
        y_voorspelling = y_norm_voorspelling * self.y_std + self.y_mean
        return y_voorspelling

    def score(self, feature, target):
        """
        Bereken de RMSE van het model met de meegegeven data.

        Parameters:
        feature (pd.DataFrame): DataFrame met 1 feature-kolom
        target (np.ndarray): Numpy array met target-kolom

        Returns:
        float: De RMSE.
        """
        y_voorspellingen = self.predict(feature)
        errors = y_voorspellingen - target
        rmse = np.sqrt(np.mean(errors ** 2))
        return rmse

## Data inladen

Ik gebruik de auto_mpg dataset om mee te testen. Ik kies ervoor om "horsepower" als feature te gebruiken, en "mpg" als target.

In [3]:
df = pd.read_csv("../databronnen/auto_mpg.csv")
df

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
387,27.0,4,140.0,86.0,2790.0,15.6,82,1,ford mustang gl
388,44.0,4,97.0,52.0,2130.0,24.6,82,2,vw pickup
389,32.0,4,135.0,84.0,2295.0,11.6,82,1,dodge rampage
390,28.0,4,120.0,79.0,2625.0,18.6,82,1,ford ranger


In [4]:
# feature en target selecteren
feature = pd.DataFrame(df["horsepower"])
target = np.array(df["mpg"])

In [5]:
# testen
my_model = MyLinearRegression()
my_model.fit(feature, target)
my_model.predict(feature)
rmse = my_model.score(feature, target)

print("w =", my_model.w)
print("b =", my_model.b)
print("RMSE:", rmse)

w = -0.7784267809047418
b = 2.7551657100901777e-17
RMSE: 4.8932262300657134


## Lineaire regressie met sklearn

In [6]:
# data
x = np.array(feature).reshape(-1, 1)  # feature moet 2D zijn
y = target

# model + fitten
model = LinearRegression()
model.fit(x, y)

# resultaten bekijken
w = model.coef_[0]
b = model.intercept_

print("w =", w)
print("b =", b)

# voorspellingen
y_voorspellingen = model.predict(x)

rmse = root_mean_squared_error(y, y_voorspellingen)
print("RMSE:", rmse)

w = -0.1578447333536537
b = 39.935861021170474
RMSE: 4.893226230065713
